<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/CSV_to_UVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier wird mit Hilfe der Messdaten der UVI berechnet

In [32]:
# Import der benötigten Module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
from google.colab import files

In [3]:
# Mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Monat der importiert werden soll
month = '22.06'

In [5]:
# Import der CSV mit den rohen Messdaten
# Pfad zur CSV-Datei auf Google Drive
drive_path = '/content/drive/My Drive/Colab_Notebooks/CSV_Messdaten/' + month

# CSV-Datei einlesen und DataFrame erstellen
df_Messdaten = pd.read_csv(drive_path)

In [66]:
df_Messdaten

,Datetime,Datum,Uhrzeit,Messzeitpunkt,290.0,290.1,290.2,290.3,290.4,290.5,...,419.1,419.2,419.3,419.4,419.5,419.6,419.7,419.8,419.9,420.0
0,2022-06-15 07:21:00,2022-06-15,07:21:00,26460,0.0,0.0,0.0,0.0,0.0,0.0,...,0.693500,0.691910,0.685560,0.669900,0.656610,0.646640,0.640170,0.636880,0.636530,0.642410
1,2022-06-15 07:23:00,2022-06-15,07:23:00,26580,0.0,0.0,0.0,0.0,0.0,0.0,...,0.723340,0.720150,0.715980,0.697770,0.691250,0.678760,0.669920,0.663800,0.661820,0.669270
2,2022-06-15 07:25:00,2022-06-15,07:25:00,26700,0.0,0.0,0.0,0.0,0.0,0.0,...,0.724600,0.719900,0.709220,0.693840,0.686640,0.676700,0.666200,0.661930,0.661600,0.671780
3,2022-06-15 07:27:00,2022-06-15,07:27:00,26820,0.0,0.0,0.0,0.0,0.0,0.0,...,0.739360,0.737440,0.729150,0.709380,0.700580,0.692580,0.680470,0.672850,0.671560,0.675320
4,2022-06-15 07:29:00,2022-06-15,07:29:00,26940,0.0,0.0,0.0,0.0,0.0,0.0,...,0.741280,0.741840,0.731630,0.710630,0.701250,0.692140,0.682440,0.676410,0.677050,0.685110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7303,2022-06-30 18:49:00,2022-06-30,18:49:00,67740,0.0,0.0,0.0,0.0,0.0,0.0,...,0.017621,0.017573,0.017332,0.016969,0.016789,0.016625,0.016391,0.016304,0.016265,0.016429
7304,2022-06-30 18:51:00,2022-06-30,18:51:00,67860,0.0,0.0,0.0,0.0,0.0,0.0,...,0.015729,0.015730,0.015471,0.015085,0.014909,0.014752,0.014533,0.014590,0.014665,0.014806
7305,2022-06-30 18:53:00,2022-06-30,18:53:00,67980,0.0,0.0,0.0,0.0,0.0,0.0,...,0.015218,0.015181,0.014969,0.014715,0.014497,0.014267,0.014077,0.013996,0.014004,0.014091
7306,2022-06-30 18:55:00,2022-06-30,18:55:00,68100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.017840,0.017820,0.017549,0.017234,0.016933,0.016672,0.016487,0.016417,0.016472,0.016619


In [ ]:
# Benutzerdefinierte Gewichtungsfunktion
def weight_function(Wellenlaenge):
    if Wellenlaenge <= 298:
        return 1
    elif Wellenlaenge > 298 and Wellenlaenge <= 328:
        return 10**(0.094 * (298.0 - Wellenlaenge))
    elif Wellenlaenge > 328 and Wellenlaenge <= 400:
        return 10**(0.015 * (139.0 - Wellenlaenge))
    else:
        return 0

In [101]:
# Anwenden der Gewichtungsfunktion auf den DataFrame
# Bereich von Spalte x bis Spalte y (einschließlich y)
x = '290.0'
y = '400.0'
selected_columns = df_Messdaten.columns[(df_Messdaten.columns.get_loc(x)):(df_Messdaten.columns.get_loc(y)+1)]

In [102]:
df_Gewichtet = []
df_Gewichtet = df_Messdaten[selected_columns].apply(lambda col: col * weight_function(float(col.name)), axis=0)

In [103]:
def integrate_row(row):
    x_values = np.array([float(col) for col in row.index])  # X-Werte aus Spaltennamen extrahieren
    y_values = np.array(row)
    integral = np.trapz(y_values, x_values)
    return integral

In [104]:
ser_UVI = []
df_UVI = []

ser_UVI = df_Gewichtet.apply(integrate_row, axis=1)
df_UVI = pd.DataFrame(ser_UVI, columns=['UVI'])

In [105]:
type(df_Gewichtet), type(df_UVI)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [106]:
# Exportieren
columns_to_copy = ['Messzeitpunkt', 'Uhrzeit', 'Datum', 'Datetime']

for col in columns_to_copy:
  df_Gewichtet.insert(0, col, df_Messdaten[col])
  df_UVI.insert(0, col, df_Messdaten[col])

In [107]:
df_Gewichtet

,Datetime,Datum,Uhrzeit,Messzeitpunkt,290.0,290.1,290.2,290.3,290.4,290.5,...,399.1,399.2,399.3,399.4,399.5,399.6,399.7,399.8,399.9,400.0
0,2022-06-15 07:21:00,2022-06-15,07:21:00,26460,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000078,0.000078,0.000077,0.000077,0.000076,0.000076,0.000076,0.000077,0.000078,0.000078
1,2022-06-15 07:23:00,2022-06-15,07:23:00,26580,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000081,0.000081,0.000080,0.000080,0.000080,0.000080,0.000080,0.000080,0.000081,0.000081
2,2022-06-15 07:25:00,2022-06-15,07:25:00,26700,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000081,0.000081,0.000081,0.000080,0.000080,0.000079,0.000079,0.000080,0.000081,0.000081
3,2022-06-15 07:27:00,2022-06-15,07:27:00,26820,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000082,0.000082,0.000082,0.000081,0.000081,0.000080,0.000080,0.000081,0.000082,0.000082
4,2022-06-15 07:29:00,2022-06-15,07:29:00,26940,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000082,0.000083,0.000083,0.000082,0.000081,0.000081,0.000081,0.000081,0.000082,0.000083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7303,2022-06-30 18:49:00,2022-06-30,18:49:00,67740,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
7304,2022-06-30 18:51:00,2022-06-30,18:51:00,67860,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
7305,2022-06-30 18:53:00,2022-06-30,18:53:00,67980,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
7306,2022-06-30 18:55:00,2022-06-30,18:55:00,68100,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
